<a href="https://colab.research.google.com/github/pterodactylus42/podcastReGenerator/blob/main/transcribe_german.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Audio-Bearbeitung

In [1]:
!pip install SpeechRecognition

In [2]:
import os

In [3]:
transcript_path = 'transcript.txt'

try:
    os.remove(transcript_path)
except FileNotFoundError:
    # doesn't exist
    print(f"{transcript_path} does not exist")

In [4]:
import speech_recognition as sr

def transcribe_audio(file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio, language='de-DE')
        print("Transcript: {}".format(text))
        with open(transcript_path, 'a') as file:
            file.write(text + "\n")
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))



In [5]:
# optional upload from your computer
# from google.colab import files
# uploaded = files.upload()

In [6]:
# assign uploaded file to filename
# for filename in uploaded.keys():
#     print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

In [7]:
import requests

url = 'https://yourfile.mp3'
response = requests.get(url)

filename = 'file.mp3'

with open(filename, 'wb') as file:
    file.write(response.content)

print('Download complete!')

Download complete!


In [8]:
!pip install pydub

In [9]:
from pydub import AudioSegment

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    # Load the MP3 file
    audio = AudioSegment.from_mp3(mp3_file_path)

    # Export as WAV
    audio.export(wav_file_path, format="wav")
    print(f"Successfully converted {mp3_file_path} to {wav_file_path}")


In [10]:
convert_mp3_to_wav(filename, "output.wav")

Successfully converted file.mp3 to output.wav


In [11]:
def process_wav_in_chunks(audiofile, chunk_length_ms=60000):
    # Load the WAV file
    audio = AudioSegment.from_wav(audiofile)

    # Calculate the number of chunks
    num_chunks = len(audio) // chunk_length_ms
    interleave = 3000

    for i in range(num_chunks + 1):
        start_time = i * chunk_length_ms
        end_time = start_time + chunk_length_ms

        interleave_start_time = start_time
        if i > 0 :
          interleave_start_time = start_time - interleave

        print(f"Processing chunk {i} of {num_chunks} from {interleave_start_time}ms to {end_time}ms")
        chunk = audio[interleave_start_time:end_time]
        chunk.export(f"chunk_{i}.wav", format="wav")
        transcribe_audio(f"chunk_{i}.wav")

    for i in range(num_chunks + 1):
        try:
            os.remove(f"chunk_{i}.wav")
        except FileNotFoundError:
            # doesn't exist
            print(f"chunk_{i}.wav does not exist")


In [ ]:
process_wav_in_chunks("output.wav")

Processing chunk 0 of 52 from 0ms to 60000ms
Processing chunk 12 of 52 from 717000ms to 780000ms


In [ ]:
# Open the file in read mode and print its contents
with open(transcript_path, 'r') as file:
    content = file.read()
    print(content)


In [ ]:
def get_last_words(line):
    words = line.split()
    return words[-3:]

In [ ]:
def get_first_words(line):
    words = line.split()
    return words[:10]

In [ ]:
transcript_stripped_path = 'transcript_stripped.txt'

try:
    os.remove(transcript_stripped_path)
except FileNotFoundError:
    # doesn't exist
    print(f"{transcript_stripped_path} does not exist")

In [ ]:
# for each line, take the last token and find it at the beginning of the next line
# the beginning of the line are the first 10 tokens
# delete everything before the occurrence of the token in the next line
# if the last token of the line is not found, search similarly with the preceding token
# try this with the last three words of the previous line
with open(transcript_path) as file_in:
    lines = []
    previousline = ""

    for line in file_in:
        # print('processing line ' + line)
        lastwords = get_last_words(previousline)
        firstwords = get_first_words(line)

        for word in lastwords:
            if firstwords.count(word) > 0 :
                index = line.find(word)
                line = line[index + len(word):]
                # print('match on word : ' + word)
                # print('resulting line : ' + line)

        lines.append(line)
        with open(transcript_stripped_path, 'a') as file:
            file.write(line + "\n")
        # print('processed  line ' + line)

        previousline = line


In [ ]:
# Open the file that is stripped from the overlapping words in read mode and print its contents
with open(transcript_stripped_path, 'r') as file:
    content = file.read()
    print(content)